In [ ]:
import sys
import os
import json
import random
import re
import socket
import threading
import time

HOST = '127.0.0.1'  # localhost
PORT = 65432

def make_emotion_json(filename):
    data = {
        "joy": round(random.uniform(0, 1), 3),
        "sadness": round(random.uniform(0, 1), 3),
        "surprise": round(random.uniform(0, 1), 3),
        "anger": round(random.uniform(0, 1), 3),
        "calm": round(random.uniform(0, 1), 3)
    }
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"{filename} 파일이 생성되었습니다.")

def get_next_index():
    files = [f for f in os.listdir('.') if re.match(r'Emotion_Metadata\(\d+\)\.json', f)]
    if not files:
        return 0
    indices = [int(re.findall(r'Emotion_Metadata\((\d+)\)\.json', f)[0]) for f in files]
    return max(indices) + 1

def delete_emotion_files():
    files = [f for f in os.listdir('.') if f.startswith('Emotion_Metadata') and f.endswith('.json')]
    for f in files:
        os.remove(f)
        print(f"{f} 파일 삭제됨.")
    print("모든 Emotion_Metadata json 파일 삭제 완료.")

def handle_client(conn, addr):
    print(f'Connected by {addr}')
    try:
        while True:
            user_input = input("종료(p), 랜덤 감정 데이터 생성(l), 감정 파일 전체 삭제(d): ")

            if user_input.lower() == 'p' or 'd':
                if user_input.lower() == 'd':
                    delete_emotion_files()
                print("프로그램을 종료합니다.")
                conn.close()
                sys.exit()

            elif user_input.lower() == 'l':
                idx = get_next_index()
                filename = f"Emotion_Metadata({idx}).json"
                make_emotion_json(filename)
                timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

                # JSON 파일명과 시간 정보를 JSON 문자열로 만들어 전송
                send_data = json.dumps({"filename": filename, "time": timestamp})
                conn.sendall(send_data.encode('utf-8'))
                print(f"데이터 전송됨: {send_data}")

    except Exception as e:
        print(f"클라이언트 통신 오류: {e}")
        conn.close()

def start_server():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()
        print(f"서버가 {HOST}:{PORT} 에서 대기 중입니다.")
        conn, addr = s.accept()
        handle_client(conn, addr)

if __name__ == "__main__":
    start_server()


OSError: [WinError 10048] 각 소켓 주소(프로토콜/네트워크 주소/포트)는 하나만 사용할 수 있습니다